In [ ]:
# idk where to put this lol
# also im just messing around here

In [1]:
import traceback
import inspect

In [18]:
x = 5
def ahhh(y):
    x = y + 4
    #print(dir())
    #print(inspect.trace())
    #print(inspect.stack())
    print(inspect.currentframe().f_back)
    print(inspect.currentframe())
    return 8
#print(globals())
#print(dir())
#print(locals())
#ahhh(8)
#traceback.extract_tb()
#frame_infos = inspect.trace()
#print(inspect.trace())
ahhh(8)
#print(len(inspect.stack()))
#print(inspect.stack())
print(inspect.currentframe())
aaa = inspect.getframeinfo(inspect.currentframe())
print(aaa)
print(inspect.getinnerframes(aaa))

<frame at 0x1107b3d80, file '/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_39313/229811572.py', line 17, code <module>>
<frame at 0x110721480, file '/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_39313/229811572.py', line 8, code ahhh>
<frame at 0x11087c250, file '/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_39313/229811572.py', line 20, code <module>>
Traceback(filename='/var/folders/8l/7g3122r17kv266mlj9yyt2zm0000gq/T/ipykernel_39313/229811572.py', lineno=21, function='<module>', code_context=['aaa = inspect.getframeinfo(inspect.currentframe())\n'], index=0, positions=Positions(lineno=21, end_lineno=21, col_offset=6, end_col_offset=50))


AttributeError: 'Traceback' object has no attribute 'f_lineno'